In [26]:
import astroplan
from astropy.coordinates import ICRS, SkyCoord, AltAz, get_moon, EarthLocation, get_body
from astropy import units as u
from astropy.utils.data import download_file
from astropy.table import Table, QTable, join
from astropy.time import Time, TimeDelta
from astropy_healpix import *
from ligo.skymap import plot
from ligo.skymap.io import read_sky_map
import healpy as hp
import os
from matplotlib import pyplot as plt
import numpy as np
from tqdm.auto import tqdm
import datetime as dt
import pickle
import pandas as pd
from docplex.mp.model import Model

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
warnings.simplefilter('ignore', astroplan.TargetNeverUpWarning)
warnings.simplefilter('ignore', astroplan.TargetAlwaysUpWarning)

In [27]:
directory_path = "/u/ywagh/test_skymaps/"
filelist = sorted([f for f in os.listdir(directory_path) if f.endswith('.gz')])

In [28]:
slew_speed = 2.5 * u.deg / u.s
slew_accel = 0.4 * u.deg / u.s**2
readout = 8.2 * u.s

In [29]:
ns_nchips = 4
ew_nchips = 4
ns_npix = 6144
ew_npix = 6160
plate_scale = 1.01 * u.arcsec
ns_chip_gap = 0.205 * u.deg
ew_chip_gap = 0.140 * u.deg

ns_total = ns_nchips * ns_npix * plate_scale + (ns_nchips - 1) * ns_chip_gap
ew_total = ew_nchips * ew_npix * plate_scale + (ew_nchips - 1) * ew_chip_gap

rcid = np.arange(64)

chipid, rc_in_chip_id = np.divmod(rcid, 4)
ns_chip_index, ew_chip_index = np.divmod(chipid, ew_nchips)
ns_rc_in_chip_index = np.where(rc_in_chip_id <= 1, 1, 0)
ew_rc_in_chip_index = np.where((rc_in_chip_id == 0) | (rc_in_chip_id == 3), 0, 1)

ew_offsets = ew_chip_gap * (ew_chip_index - (ew_nchips - 1) / 2) + ew_npix * plate_scale * (ew_chip_index - ew_nchips / 2) + 0.5 * ew_rc_in_chip_index * plate_scale * ew_npix
ns_offsets = ns_chip_gap * (ns_chip_index - (ns_nchips - 1) / 2) + ns_npix * plate_scale * (ns_chip_index - ns_nchips / 2) + 0.5 * ns_rc_in_chip_index * plate_scale * ns_npix

ew_ccd_corners = 0.5 * plate_scale * np.asarray([ew_npix, 0, 0, ew_npix])
ns_ccd_corners = 0.5 * plate_scale * np.asarray([ns_npix, ns_npix, 0, 0])

ew_vertices = ew_offsets[:, np.newaxis] + ew_ccd_corners[np.newaxis, :]
ns_vertices = ns_offsets[:, np.newaxis] + ns_ccd_corners[np.newaxis, :]

def get_footprint(center):
    return SkyCoord(
        ew_vertices, ns_vertices,
        frame=center[..., np.newaxis, np.newaxis].skyoffset_frame()
    ).icrs

url = 'https://github.com/ZwickyTransientFacility/ztf_information/raw/master/field_grid/ZTF_Fields.txt'
filename = download_file(url)
field_grid = QTable(np.recfromtxt(filename, comments='%', usecols=range(3), names=['field_id', 'ra', 'dec']))
field_grid['coord'] = SkyCoord(field_grid.columns.pop('ra') * u.deg, field_grid.columns.pop('dec') * u.deg)
field_grid = field_grid[0:881]   #working only with primary fields

In [30]:
#******************************************************************************
skymap, metadata = read_sky_map(os.path.join(directory_path, filelist[7]))

plot_filename = os.path.basename(filelist[7])
#******************************************************************************

In [31]:
event_time = Time(metadata['gps_time'], format='gps').utc
event_time.format = 'iso'
print('event time:',event_time)
observer = astroplan.Observer.at_site('Palomar')
night_horizon = -18 * u.deg
if observer.is_night(event_time, horizon=night_horizon):
    start_time = event_time
else:
    start_time = observer.sun_set_time(
        event_time, horizon=night_horizon, which='next')

# Find the latest possible end time of observations: the time of sunrise.
end_time = observer.sun_rise_time(
    start_time, horizon=night_horizon, which='next')

min_airmass = 2.5 * u.dimensionless_unscaled
airmass_horizon = (90 * u.deg - np.arccos(1 / min_airmass))
targets = field_grid['coord']

# Find the time that each field rises and sets above an airmass of 2.5.
target_start_time = Time(np.where(
    observer.target_is_up(start_time, targets, horizon=airmass_horizon),
    start_time,
    observer.target_rise_time(start_time, targets, which='next', horizon=airmass_horizon)))
target_start_time.format = 'iso'

# Find the time that each field sets below the airmass limit. If the target
# is always up (i.e., it's circumpolar) or if it sets after surnsise,
# then set the end time to sunrise.
target_end_time = observer.target_set_time(
    target_start_time, targets, which='next', horizon=airmass_horizon)
target_end_time[
    (target_end_time.mask & ~target_start_time.mask) | (target_end_time > end_time)
] = end_time
target_end_time.format = 'iso'
# Select fields that are observable for long enough for at least one exposure
##############################################################################
exposure_time = 30 * u.second
##############################################################################
field_grid['start_time'] = target_start_time
field_grid['end_time'] = target_end_time
observable_fields = field_grid[target_end_time - target_start_time >= exposure_time]

# print(observable_fields)
hpx = HEALPix(nside=256, frame=ICRS())

footprint = np.moveaxis(
    get_footprint(SkyCoord(0 * u.deg, 0 * u.deg)).cartesian.xyz.value, 0, -1)
footprint_healpix = np.unique(np.concatenate(
    [hp.query_polygon(hpx.nside, v, nest=(hpx.order == 'nested')) for v in footprint]))

'''
# computing the footprints of every ZTF field as HEALPix indices. Downsampling skymap to same resolution.
'''
footprints = np.moveaxis(get_footprint(observable_fields['coord']).cartesian.xyz.value, 0, -1)
footprints_healpix = [
    np.unique(np.concatenate([hp.query_polygon(hpx.nside, v) for v in footprint]))
    for footprint in tqdm(footprints)]

prob = hp.ud_grade(skymap, hpx.nside, power=-2)

# k = max number of 300s exposures 
min_start = min(observable_fields['start_time'])
max_end =max(observable_fields['end_time'])
min_start.format = 'jd'
max_end.format = 'jd'
k = int(np.floor((max_end - min_start)/(2*exposure_time.to(u.day))))
print(k," number of exposures could be taken tonight")

print("problem setup completed")

event time: 2019-12-12 08:27:28.643


/u/ywagh/.local/lib/python3.10/site-packages/astropy/units/quantity.py:671: RuntimeWarning: invalid value encountered in divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


  0%|          | 0/398 [00:00<?, ?it/s]

285  number of exposures could be taken tonight
problem setup completed


## Model one

In [32]:
m1 = Model('max coverage problem')

field_vars = m1.binary_var_list(len(footprints), name='field')
pixel_vars = m1.binary_var_list(hpx.npix, name='pixel')

footprints_healpix_inverse = [[] for _ in range(hpx.npix)]

for field, pixels in enumerate(footprints_healpix):
    for pixel in pixels:
        footprints_healpix_inverse[pixel].append(field)

for i_pixel, i_fields in enumerate(footprints_healpix_inverse):
     m1.add_constraint(m1.sum(field_vars[i] for i in i_fields) >= pixel_vars[i_pixel])

m1.add_constraint(m1.sum(field_vars) <= k)
m1.maximize(m1.dot(pixel_vars, prob))
print("number fo fields observed should be less than k")

solution = m1.solve(log_output=True)

print("optimization completed")
total_prob_covered = solution.objective_value

print("Total probability covered:",total_prob_covered)


number fo fields observed should be less than k
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.03 sec. (24.20 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 786335 rows and 729253 columns.
Aggregator did 98 substitutions.
All rows and columns eliminated.
Presolve time = 0.48 sec. (531.67 ticks)

Root node processing (before b&c):
  Real time             =    0.54 sec. (599.14 ticks)
Parallel b&c, 32 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.54 sec. (599.14 ticks)
optimization completed
Total probability covered: 0.7935663331539322


In [33]:
selected_fields_ID = [i for i, v in enumerate(field_vars) if v.solution_value == 1]
# print(selected_fields_ID)
selected_fields = observable_fields[selected_fields_ID]


separation_matrix = selected_fields['coord'][:,np.newaxis].separation(selected_fields['coord'][np.newaxis,:])

def slew_time(separation):
   return np.where(
       separation <= (slew_speed**2 / slew_accel), 
       np.sqrt(2 * separation / slew_accel), 
       (2 * slew_speed / slew_accel) + (separation - slew_speed**2 / slew_accel) / slew_speed
       )

slew_times = slew_time(separation_matrix).value

slew_time_value = slew_times*u.second
slew_time_day = slew_time_value.to_value(u.day)

# slew_time_max = np.max(slew_times) * u.second
# slew_time_max_ = slew_time_max.to_value(u.day) #test

## model two

In [34]:
# m2 = Model("Telescope timings")

# observer_location = EarthLocation.of_site('Palomar')

# #calculate the probability for all the selected fields
# footprints_selected = np.moveaxis(get_footprint(selected_fields['coord']).cartesian.xyz.value, 0, -1)
# footprints_healpix_selected = [
#     np.unique(np.concatenate([hp.query_polygon(hpx.nside, v) for v in footprint]))
#     for footprint in tqdm(footprints_selected)]

# probabilities = []

# for field_index in range(len(footprints_healpix_selected)):
#     probability_field = np.sum(prob[footprints_healpix_selected[field_index]])
#     probabilities.append(probability_field)
# print("worked for",len(probabilities),"fields")

# selected_fields['probabilities'] = probabilities

# delta = exposure_time.to_value(u.day)
# M = (selected_fields['end_time'].max() - selected_fields['start_time'].min()).to_value(u.day).item()
# M = M * 20
# num_visits = 3
# num_filters = 2
# # cadence = 90/(60*24)
# # cadence = (cadence_minutes * u.minute).to(u.day).value

# x = [m2.binary_var(name=f"x_{i}") for i in range(len(selected_fields))]

# s = [[m2.binary_var(name=f's_{i}_{j}') for j in range(i)] for i in range(len(selected_fields))]

# # tc = [[[m2.continuous_var(
# #             lb=(row['start_time'] - start_time).to_value(u.day),
# #             ub=(row['end_time'] - start_time - exposure_time).to_value(u.day),
# #             name=f"start_time_field_{i}_filter_{j}_visit_{k}"
# #         ) for k in range(num_visits)] for j in range(num_filters)] for i, row in enumerate(selected_fields)]

# tc = [[m2.continuous_var(
#     lb =(row['start_time'] - start_time).to_value(u.day),
#     ub=(row['end_time'] - start_time - exposure_time).to_value(u.day),
#     name=f"start_time_field_{i}_visit_{k}")
#        for k in range(num_visits*num_filters)] for i,row in enumerate(selected_fields)]

# # Add constraints for cadence
# cadence_minutes = 5
# cadence_days = cadence_minutes / (60 * 24)



# # for v in range(1,num_visits):
# #     for f1 in range(num_filters):
# #         for f2 in range(num_filters):
# #             for i in range(len(selected_fields)):
# #                 m2.add_constraint(tc[i][f1][v]-tc[i][f2][v-1] >= cadence_days*x[i], ctname = f"cadence_constraint_field_{i}_filter_{f1}{f2}_visit_{v}")

# for v in range(1, num_visits*num_filters):
#     for i in range(len(selected_fields)):
#         m2.add_constraint(tc[i][v]-tc[i][v-1] >= cadence_days * x[i], ctname = f"cadence_constraint_feild_{i}_visits_{v}")

# # for v1 in range(num_visits):
# #     for v2 in range(num_visits):
# #         for f1 in range(num_filters):
# #             for f2 in range(num_filters):
# #                 for i in range(len(selected_fields)):
# #                     for j in range(i):
# #                         m2.add_constraint(tc[i][f1][v1] + delta * x[i] + slew_time_day[i][j] - tc[j][f2][v2] <= M * (1 - s[i][j]),
# #                             ctname=f"non_overlap1_for_field_{i}_and_{j}_filter_{f1}_visit_{v1}_to_filter_{f2}_visit_{v2}")
# #                         m2.add_constraint(tc[j][f2][v2] + delta * x[j] + slew_time_day[i][j] - tc[i][f1][v1] <= M * s[i][j],
# #                             ctname=f"non_overlap2_for_field_{i}_and_{j}_filter_{f2}_visit_{v2}_to_filter_{f1}_visit_{v1}")

# for v in range(1,num_visits*num_filters):
#     for i in range(len(selected_fields)):
#         for j in range(i):
#             m2.add_constraint(tc[i][v-1] + delta * x[i] +slew_time_day[i][j] - tc[j][v] <= M * (1 - s[i][j]),
#                               ctname = f"non_overlapping1_fileds_{i}_{j}_visits_{v-1}_{v}")
#             m2.add_constraint(tc[j][v] + delta * x[j] +slew_time_day[i][j] - tc[i][v-1] <= M * s[i][j],
#                               ctname = f"non_overlapping2_fileds_{i}_{j}_visits_{v-1}_{v}")
# #visit sequencing
# # for v in range(1, num_visits):
# #     for f in range(num_filters):
# #         for i in range(len(selected_fields)):
# #             for j in range(j):
# #                 m2.add_constraint(tc[i][f][v-1] + delta * x[i] + slew_time_day[i][j] <= tc[i][f][v],
# #                                   ctname=f"visit_sequncing_for_field_{i}for_filter{f}to_visit{v}")

# m2.maximize(m2.sum([probabilities[i] * x[i] for i in range(len(selected_fields))]))

# # Solve
# m2.parameters.timelimit = 120
# solution = m2.solve(log_output=True)

# # Extract scheduled times
# # scheduled_start_times = [[[solution.get_value(tc[i][f][v]) for v in range(num_visits)] for f in range(num_filters)] for i in range(len(selected_fields))]
# # print(scheduled_start_times)


In [35]:
m2 = Model("Telescope timings")

observer_location = EarthLocation.of_site('Palomar')

footprints_selected = np.moveaxis(get_footprint(selected_fields['coord']).cartesian.xyz.value, 0, -1)
footprints_healpix_selected = [
    np.unique(np.concatenate([hp.query_polygon(hpx.nside, v) for v in footprint]))
    for footprint in tqdm(footprints_selected)]

probabilities = []

for field_index in range(len(footprints_healpix_selected)):
    probability_field = np.sum(prob[footprints_healpix_selected[field_index]])
    probabilities.append(probability_field)
print("worked for",len(probabilities),"fields")

selected_fields['probabilities'] = probabilities

delta = exposure_time.to_value(u.day)
M = (selected_fields['end_time'].max() - selected_fields['start_time'].min()).to_value(u.day).item()
M = M * 20
num_visits = 2
num_filters = 2

x = [m2.binary_var(name=f"x_{i}") for i in range(len(selected_fields))]

s = [[m2.binary_var(name=f's_{i}_{j}') for j in range(i)] for i in range(len(selected_fields))]

tc = [[m2.continuous_var(
    lb =(row['start_time'] - start_time).to_value(u.day),
    ub=(row['end_time'] - start_time - exposure_time).to_value(u.day),
    name=f"start_time_field_{i}_visit_{k}")
       for k in range(num_visits*num_filters)] for i,row in enumerate(selected_fields)]

cadence_minutes = 30
cadence_days = cadence_minutes / (60 * 24)

for v in range(1, num_visits*num_filters):
    for i in range(len(selected_fields)):
        m2.add_constraint(tc[i][v]-tc[i][v-1] >= cadence_days * x[i], ctname = f"cadence_constraint_feild_{i}_visits_{v}")

for v in range(1,num_visits*num_filters):
    for i in range(len(selected_fields)):
        for j in range(i):
            m2.add_constraint(tc[i][v-1] + delta * x[i] +slew_time_day[i][j] - tc[j][v] <= M * (1 - s[i][j]),
                              ctname = f"non_overlapping1_fileds_{i}_{j}_visits_{v-1}_{v}")
            m2.add_constraint(tc[j][v] + delta * x[j] +slew_time_day[i][j] - tc[i][v-1] <= M * s[i][j],
                              ctname = f"non_overlapping2_fileds_{i}_{j}_visits_{v-1}_{v}")

m2.maximize(m2.sum([probabilities[i] * x[i] for i in range(len(selected_fields))]))

m2.parameters.timelimit = 120
solution = m2.solve(log_output=True)

  0%|          | 0/100 [00:00<?, ?it/s]

worked for 100 fields
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               120
Tried aggregator 1 time.
MIP Presolve eliminated 2788 rows and 506 columns.
MIP Presolve modified 46724 coefficients.
Reduced MIP has 27212 rows, 4944 columns, and 98820 nonzeros.
Reduced MIP has 4546 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.05 sec. (58.81 ticks)
Found incumbent of value 0.000000 after 0.09 sec. (101.84 ticks)
Probing time = 0.02 sec. (6.26 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve modified 52 coefficients.
Reduced MIP has 27212 rows, 4944 columns, and 98820 nonzeros.
Reduced MIP has 4546 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.05 sec. (45.31 ticks)
Probing time = 0.01 sec. (6.26 ticks)
Clique table members: 1709.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: det

In [ ]:
# Get the indices of scheduled fields
scheduled_fields_ID = [i for i, v in enumerate(x) if v.solution_value == 1]
scheduled_fields = selected_fields[scheduled_fields_ID]
# scheduled_fields
scheduled_tc = [[solution.get_value(tc[i][v]) for v in range(num_visits * num_filters)] for i in scheduled_fields_ID]
scheduled_tc = np.asarray(scheduled_tc)

In [52]:
for i in range(num_visits*num_filters):
    scheduled_fields[f"Scheduled_start_filt_times_{i}"] = scheduled_tc[:,i]


In [53]:
scheduled_fields

field_id,coord,start_time,end_time,probabilities,Scheduled_start_filt_times_0,Scheduled_start_filt_times_1,Scheduled_start_filt_times_2,Scheduled_start_filt_times_3
,"deg,deg",,,,,,,
int64,SkyCoord,Time,Time,float64,float64,float64,float64,float64
404,"57.9375,-2.65",2019-12-12 08:27:28.643,2019-12-12 10:14:28.357,1.0584760518986203e-07,0.0006785035088915639,0.021511836842224897,0.04234517017555823,0.06317850350889156
450,"26.27176,4.55",2019-12-12 08:27:28.643,2019-12-12 13:13:05.080,0.010574655798101344,0.0007377113677011316,0.021571044701034464,0.04240437803436779,0.06323771136770112
454,"54.31528,4.55",2019-12-12 08:27:28.643,2019-12-12 10:21:15.351,0.00010909356538981048,0.0006838259787879746,0.021517159312121305,0.05169530152888645,0.07253268859058219
455,"61.32616,4.55",2019-12-12 08:27:28.643,2019-12-12 10:49:11.620,1.2416955271372923e-07,0.0007085609696095601,0.021541894302942892,0.04237522763627623,0.06320856096960956
506,"50.66337,11.75",2019-12-12 08:27:28.643,2019-12-12 10:25:33.352,0.0021224843923198676,0.0020801095505161753,0.022913442883849507,0.043746776217182835,0.06458010955051617
507,"57.64762,11.75",2019-12-12 08:27:28.643,2019-12-12 10:53:23.826,0.00012277317272538297,0.0021081376591124412,0.022941470992445773,0.043774804325779106,0.06460813765911244
508,"64.63187,11.75",2019-12-12 08:27:28.643,2019-12-12 11:21:14.077,5.181175347451236e-08,0.0021797212923037952,0.023013054625637127,0.04384638795897046,0.06467972129230379
556,"44.87418,18.95",2019-12-12 08:27:28.643,2019-12-12 10:19:50.403,0.02677156724154399,0.002082319051762485,0.022915652385095815,0.04374898571842915,0.06458231905176248
